In [1]:
import tensorflow as tf

In [2]:
tf.constant([[1,2,3], [4,5,6]])

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [3]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
t = tf.constant([[1,2,3], [4,5,6]])
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.int32

In [6]:
type(t)

tensorflow.python.framework.ops.EagerTensor

In [7]:
t[:, 1:2]

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[2],
       [5]], dtype=int32)>

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[11, 12, 13],
       [14, 15, 16]], dtype=int32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 1,  4,  9],
       [16, 25, 36]], dtype=int32)>

In [10]:
tf.transpose(t)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 4],
       [2, 5],
       [3, 6]], dtype=int32)>

In [11]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[14, 32],
       [32, 77]], dtype=int32)>

In [12]:
import numpy as np

In [13]:
a = np.array([2,4.5,5])
t = tf.constant(a)
t

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2. , 4.5, 5. ])>

In [14]:
t.numpy()

array([2. , 4.5, 5. ])

In [15]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4.  , 20.25, 25.  ])>

In [16]:
np.square(t)

array([ 4.  , 20.25, 25.  ])

In [17]:
tf.constant(1) + tf.constant(1.1)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a int32 tensor but is a float tensor [Op:AddV2] name: 

In [ ]:
v = tf.Variable([[1.,2,3], [4,5,6]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [ ]:
v.assign(v * 2)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [ ]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [ ]:
v[:,:2].assign(v[:,:2] * 2)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 84.,  6.],
       [16., 20., 12.]], dtype=float32)>

In [ ]:
v.scatter_nd_update(indices=[[0,0]], updates=[100.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  84.,   6.],
       [ 16.,  20.,  12.]], dtype=float32)>

In [112]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.squared(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [107]:
precision = tf.keras.metrics.Precision()

In [108]:
precision([1,0,0], [1,0,1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [110]:
precision([1,0,0,1,0,0,1,0,0,1,0,0,], [0,1,1,0,1,1,0,1,1,0,1,1,])

<tf.Tensor: shape=(), dtype=float32, numpy=0.10000000149011612>

In [111]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.10000000149011612>

In [115]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.huber_fn = huber_fn
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="elu", kernel_initializer="he_normal") for _ in range(n_layers)]
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [ ]:
class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2,30)
        self.block2 = ResidualBlock(2,30)
        self.block2 = ResidualBlock(2,30)
        
    def call(self,inputs):
        Z = self.hidden1(inputs)
        for _ in range(1+3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)
            

In [116]:
def f(x,y):
    return 3*x**2 + 2*x*y

In [121]:
eps = 1e-6
(f(5+eps, 3) - f(5,3))/eps

36.000003007075065

In [122]:
(f(5, 3+eps) - f(5,3))/eps

10.000000003174137

In [127]:
x, y = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(x,y)
    
gradients = tape.gradient(z, [x,y])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [ ]:
from tensorflow import keras

In [6]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)

In [3]:
l2_reg = tf.keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal", kernel_regularizer=l2_reg),
    keras.layers.Dense(1,kernel_regularizer=l2_reg)
])


In [15]:

def random_batch(X,y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [5]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)

In [18]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(0.01)
loss_fn = keras.losses.MeanSquaredError()
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]
import numpy as np

for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_state()

Epoch 1/5
15480/15480 - mean: 311.057678 - mean_absolute_error: 4.330317
Epoch 2/5
15480/15480 - mean: 3.222724 - mean_absolute_error: 0.870835
Epoch 3/5
15480/15480 - mean: 60.558750 - mean_absolute_error: 1.738670
Epoch 4/5
15480/15480 - mean: 33.400864 - mean_absolute_error: 1.512011
Epoch 5/5
15480/15480 - mean: 67.393700 - mean_absolute_error: 2.094926


In [19]:
def cube(x):
    return x ** 3

cube(2)

8

In [21]:
cube(tf.constant(2))

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [ ]:
tf_cube = tf.function(cube)
tf_cube

In [23]:
@tf.function
def cube(x):
    return x ** 3
cube

In [26]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [27]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>